# Автомодерация коротких текстов с математическими выражениями

В тетрадке демонстрируется возможность модерации текста, содержащего математические сущности: корректная разметка на языке LaTeX считается эталоном качества и проходит модерацию в то время, как математические выражения, заданные с помощью словесных формулировок, математических символом Unicode или языка разметки AsciiMath, – нет.

In [ ]:
from IPython.display import display, HTML

from emma import Moderator

## Создание модератора

Валидацию входного текста, содержащего математические выражения, выполняет объект типа `Moderator`:

In [3]:
m = Moderator()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/841 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.22k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/958 [00:00<?, ?B/s]

По умолчанию объект модератора инкапсулирует модель `basic-go/texbert-moderator` и упрощает взаимодействие с ней.

При инициализации объекта модератора с помощью параметра `device` можно задать устройство, на котором будут выполняться вычисления.

### Обработка выражений, содержащих математических символов Unicode

Объект класса `Moderator` обладает методов `moderate`, возвращающим `True`, если входной текст проходит модерацию, и `False` в противном случае.

In [4]:
input_text = "1/α + 1/β = 1/γ"

m.moderate(input_text)

False

Также с помощью метода `moderate_proba` предоставлена возможность получать метку и оценку уверенности классификации:

In [5]:
input_text = "1/α + 1/β = 1/γ"

m.moderate_proba(input_text)

[{'label': 'corrupted', 'score': 0.9999922513961792}]

Вместе с тем указанное выше выражение, набранное с помощью LaTeX, проходит модерацию:

In [6]:
input_text = r"$ \frac{1}{\alpha} + \frac{1}{\beta} = \frac 1 \gamma $"

display(HTML(input_text))

m.moderate(input_text)

True

### Обработка выражений, содержащих повреждения разметки на языке LaTeX

In [7]:
input_text = r"\frac{1{3 - x^{2 + \phi(x(t)"

m.moderate(input_text)

False

In [8]:
input_text = r"\( \frac{1}{3} - x^{2} + \phi(x(t)) \)"

m.moderate(input_text)

True

### Обработка выражений, содержащих разметку AsciiMath

In [9]:
input_text = "f : RR^(2) -> RR^(3)"

m.moderate(input_text)

False

In [10]:
input_text = "$$ f : \mathbb{R}^2 \to \mathbb{R}^3 $$"

m.moderate(input_text)

True

### Обработка выражений, содержащих словесные формулировки

In [11]:
input_text = "предел при эс стремящемся к минус бесконечности синус эс делить на эс плюс один"

m.moderate(input_text)

False

In [12]:
input_text = r"\( \lim_{s \to - \infty} \frac{\sin s}{s + 1} \)"

m.moderate(input_text)

True

### Другие примеры

В примере иллюстрируется набор целых чисел с тысячами.

In [13]:
input_text = "Зарплата Пети составляет 100 000 руб. Насколько его зарплата больши зарплаты Вани?"

m.moderate(input_text)

False

In [14]:
input_text = r"Зарплата Пети составляет $100\,000$ руб. Насколько его зарплата больши зарплаты Вани?"

m.moderate(input_text)

True

В этом примере демонстриуется наличие и отсутствие опечатки при использовании команды.

In [15]:
input_text = """  Рассмотрим основное тригонометрическое тождество:
\\begin{eqation}
\\sin^2 x + \\cos^2 x = 1.
\\end{equation}
"""

m.moderate(input_text)

False

In [16]:
input_text = """  Рассмотрим основное тригонометрическое тождество:
\\begin{equation}
\\sin^2 x + \\cos^2 x = 1.
\\end{equation}
"""

m.moderate(input_text)

True